In [ ]:
pip install dateparser

     |████████████████████████████████| 288 kB 1.3 MB/s eta 0:00:01
     |████████████████████████████████| 339 kB 17.5 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [ ]:
#Aqui le puse contadores visuales para ver el avance
import dateparser
import re
from datetime import date
import pandas as pd
from datetime import datetime
import requests
import sys
import json
from pandas.io.json import json_normalize
from bs4 import BeautifulSoup
import time
import random
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from webdriver_manager.chrome import ChromeDriverManager
listaid=[792104374161319]
# Colocamos los ID de los grupos de Facebook que vamos a scrapear
for i in listaid:
    dataC = pd.DataFrame(columns=('autor','comentario','post_id', 'comment_id'))
    dataP = pd.DataFrame(columns=('fanpage', 'autor', 'texto', 'post_id','time' , 'fecha', 'permalink', 'likes', 'comments', 'shares', 'image','type', 'cliente'))
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument("--incognito")
    fanpage=str(i)
    Type='groups'
    cliente='Escuelas'
    # El cliente va cambiando conforme el proyecto.
    n=10
    # Numero de scrols que va a dar nuestro bot.
    fecha = '110522'
    # Representa la fecha del día que hicimos el scrap.
    
    browser = webdriver.Chrome(ChromeDriverManager().install())
    browser.get('https://m.facebook.com/')
    username_box = browser.find_element_by_id('m_login_email')
    #Aquí poner email como str
    username_box.send_keys("rcastillo@acceso.com") 
    password_box = browser.find_element_by_id('m_login_password')
    #Aquí el password como str
    password_box.send_keys("AbrirCA2") 
    login_box = browser.find_element_by_name('login')
    login_box.click()
    time.sleep(30)

    urlm='https://m.facebook.com/'+Type+'/'+fanpage+'/'
    browser.get(urlm)
    #Vamos a bajar n veces

    for scrols in range(n): 
        ActionChains(browser).send_keys(Keys.END).perform()
        sleep = random.randint(15, 25)
        time.sleep(sleep)
        print(str(scrols)+"/"+str(n)+" scrolls")

    elemd = browser.find_element_by_xpath("//*")
    soupd = BeautifulSoup(elemd.get_attribute('innerHTML'),'html.parser')
    posts = soupd.find_all("article",'_55wo _5rgr _5gh8 async_like')
    listaTotal=[]
    listaConCom=[]
    for po in posts:
        text = str(po)
        left = ',"feedback_target":'
        right = ',"feedback_source"'
    #Vemos si tiene comments o shares
        if po.findChild("span", class_=["_1j-c"] ):
            metrica = po.find_all("span", '_1j-c')[0].get_text()
            listaConCom.append(text[text.index(left)+len(left):text.index(right)])
        listaTotal.append(text[text.index(left)+len(left):text.index(right)])
        content = po.find_all("div", '_5rgt _5nk5 _5msi')
        headerAutor = po.find_all("div",'_5xu4')
        date = po.find_all("div", '_52jc _5qc4 _78cz _24u0 _36xo')
        reactions = po.find_all("div", '_1g06')
        comshares = po.find_all("span", '_1j-c')
        images = po.find_all("div", '_5uso _5t8z')
        autor=''
        texto=''
        post_id=text[text.index(left)+len(left):text.index(right)]
        Time=''
        likes=''
        comments=''
        shares=''
        image=''
        permalink = urlm+'permalink/'+str(post_id)
        fecha=str(dateparser.parse('hoy'))[0:10]
        if headerAutor:
            for ha in headerAutor:
                if ha.find('a'):
                    autor = ha.find('a').text
        if content:
            for con in content:
                try:
                    text = con.find('p').text
                except:
                    text = 'SINTEXTO'
        if date:
            for d in date:
                fechapre = d.find('a').text
                Time = str(dateparser.parse(fechapre))[0:19]
        if reactions:
            for r in reactions:
                likes = int(r.get_text())
        if comshares:
            for cs in comshares:
                txt=cs.get_text()
                if 'Comment' in txt:
                    comments = int(re.search(r'(\d+) Comment', txt).group().strip(' Comment'))
                    if 'Share' in txt:
                        shares = int(re.search(r'(\d+) Share', txt).group().strip(' Share'))
                    else:
                        shares = 0
                else:
                    comments = 0
                    if 'Share' in txt:
                        shares = int(re.search(r'(\d+) Share', txt).group().strip(' Share'))
                    else:
                        shares = 0
        if images:
            for img in images:
                image =img.find('a')['href']
        dataP = dataP.append({'fanpage':fanpage, 'autor': autor, 'texto': text, 'post_id':post_id,'time':Time , 'fecha':fecha, 'permalink':permalink, 'likes':likes, 'comments':comments, 'shares':shares, 'image':image,'type':Type, 'cliente':cliente},ignore_index=True)


    ds = fecha
    ns = str(n)

    dataP2 = dataP
    dataP2.drop_duplicates(subset=['fecha', 'autor'])
    dataP2.to_csv(str(i)+'posts'+ds+'scroll'+ns+'.csv', encoding='UTF-8')
    listaTotal2 = list(set(listaTotal))
    listaConCom2 = list(set(listaConCom))
    print("Encontré "+str(len(listaTotal2))+"posts, "+str(len(listaConCom2))+" tenían comentarios")
    for idx, po in enumerate(listaConCom2):
        print(str(idx)+"/"+str(len(listaConCom2)))
        browser.get("https://m.facebook.com/"+po)
        elem = browser.find_element_by_xpath("//*")
        soup = BeautifulSoup(elem.get_attribute('innerHTML'),'html.parser')
        comments = soup.find_all("div",'_2b06')
        for com in comments:
            autor = com.find_all("div",'_2b05')[0].get_text()
            comentario = com.find_all("div",'')[1].get_text()
            comment_id1 = com.find_all("div",'')[1]
            comment_id = comment_id1["data-commentid"]
            dataC = dataC.append({'autor': autor,'comentario':comentario, 'post_id':po, 'comment_id':comment_id},ignore_index=True)
            sleep = random.randint(20, 60)
        time.sleep(sleep)



    ds = fecha
    ns = str(n)
    dataC2=dataC.drop_duplicates(subset=['autor', 'comentario'])
    dataC2.to_csv(str(i)+'COMMENTS'+ds+'scroll'+ns+'.csv', encoding='UTF-8')



====== WebDriver manager ======
Current google-chrome version is 101.0.4951
Get LATEST chromedriver version for 101.0.4951 google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/101.0.4951.41/chromedriver_mac64.zip
Driver has been saved in cache [/Users/raul.castillo/.wdm/drivers/chromedriver/mac64/101.0.4951.41]


0/10 scrolls
1/10 scrolls
2/10 scrolls
3/10 scrolls
4/10 scrolls
5/10 scrolls
6/10 scrolls
7/10 scrolls
8/10 scrolls
9/10 scrolls


/Users/raul.castillo/opt/anaconda3/lib/python3.8/site-packages/dateparser/date_parser.py:35: PytzUsageWarning: The localize method is no longer necessary, as this time zone supports the fold attribute (PEP 495). For more details on migrating to a PEP 495-compliant implementation, see https://pytz-deprecation-shim.readthedocs.io/en/latest/migration.html
  date_obj = stz.localize(date_obj)


Encontré 220posts, 172 tenían comentarios
0/172
1/172
2/172
3/172
4/172
5/172
6/172
7/172
8/172
9/172
10/172
11/172
12/172
13/172
14/172
15/172
16/172
17/172
18/172
19/172
20/172
21/172
22/172
23/172
24/172
25/172
26/172
27/172
28/172
29/172
30/172
31/172
32/172
33/172
34/172
35/172
36/172
37/172
38/172
39/172
40/172
41/172
42/172
43/172
44/172
45/172
46/172
47/172
48/172
49/172
50/172
51/172
52/172
53/172
54/172
55/172
56/172
57/172
58/172
59/172
60/172
61/172
62/172
63/172
64/172
65/172
66/172
67/172
68/172
69/172
70/172
71/172
72/172
73/172
74/172
75/172
76/172
77/172
78/172
79/172
80/172
81/172
82/172
83/172
84/172
85/172
86/172
87/172
88/172
89/172
90/172
91/172
92/172
93/172
94/172
95/172
96/172
97/172
98/172
99/172
100/172
101/172
102/172
103/172
104/172
105/172
106/172
107/172
108/172
109/172
110/172
111/172
112/172
113/172
114/172
115/172
116/172
117/172
118/172
119/172
120/172
121/172
122/172
123/172
124/172
125/172
126/172
127/172
128/172
129/172


WebDriverException: Message: unknown error: net::ERR_CONNECTION_CLOSED
  (Session info: chrome=101.0.4951.54)


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87c9d0b0-6942-480f-83c8-ce8875c09714' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>